In [1]:
import QuantLib as ql

calcDate = ql.Date(14, 6, 2016)
ql.Settings.instance().evaluationDate = calcDate

# Singular interest rate term structure for constant volatility.
dates = [ql.Date(14,6,2016), ql.Date(14,9,2016),
         ql.Date(14,12,2016), ql.Date(14,6,2017),
         ql.Date(14,6,2019), ql.Date(14,6,2021),
         ql.Date(15,6,2026), ql.Date(16,6,2031),
         ql.Date(16,6,2036), ql.Date(14,6,2046)]

yields = [0.000000, 0.006616, 0.007049, 0.007795,
          0.009599, 0.011203, 0.015068, 0.017583,
          0.018998, 0.020080]

dayCount = ql.ActualActual(ql.ActualActual.Bond)
calendar = ql.UnitedStates()
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual
termStructure = ql.ZeroCurve(dates, yields, dayCount, calendar, interpolation, compounding, compoundingFrequency)
ts_handle = ql.YieldTermStructureHandle(termStructure)

# Cap
start_date = ql.Date(14, 6, 2016)
end_date = ql.Date(14, 6 , 2026)
period = ql.Period(3, ql.Months)
calendar = ql.UnitedStates()
buss_convention = ql.ModifiedFollowing
rule = ql.DateGeneration.Forward
end_of_month = False
schedule = ql.Schedule(start_date, end_date, period, calendar, buss_convention, buss_convention, rule, end_of_month)

# Schedule
iborIndex = ql.USDLibor(ql.Period(3, ql.Months), ts_handle)
iborIndex.addFixing(ql.Date(10,6,2016), 0.0065560)
iborLeg = ql.IborLeg([1000000], schedule, iborIndex)

# Pricing the cap through the ibor leg and strike using BlackFloorEngine.
strike = 0.02
cap = ql.Cap(iborLeg, [strike])
vols = ql.QuoteHandle(ql.SimpleQuote(0.547295))
black_engine = ql.BlackCapFloorEngine(ts_handle, vols)
cap.setPricingEngine(black_engine) 
print("Value of Caps given constant volatility:", cap.NPV())

Value of Caps given constant volatility: 54361.469655539724
